# Day 2: Data Cleaning & Feature Engineering
## Nairobi House Prediction Project

## Objectives
1. Handle missing values
2. Remove duplicates and outliers
3. Encode categorical features
4. Create new features
5. Save processed data

## 1. Import Libraries

In [ ]:
# Data manipulation

# Preprocessing

# Visualization

## 2. Load Raw Data

In [ ]:
# Load data from raw folder

## 3. Missing Values Analysis

In [ ]:
# Check missing values

In [ ]:
# Handle missing values

## 4. Duplicate Removal

In [ ]:
# Check and remove duplicates

## 5. Outlier Detection and Treatment

In [ ]:
# Identify outliers

In [ ]:
# Handle outliers

## 6. Feature Encoding

In [ ]:
# Encode categorical variables

## 7. Feature Engineering

In [ ]:
# Create new features

## 8. Data Validation

In [ ]:
# Validate cleaned data

## 9. Save Processed Data

In [ ]:
# Save to processed folder

## Summary

### Data Quality Improvements
- [To be filled]

### Features Created
- [To be filled]

### Next Steps
- [To be filled]